In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt 
from py3gpp import *
import csv

In [2]:
## IQ Input files

# input_file = '/home/chatchamon/workarea/IQ_constellation/iq_python/input/IQDataFile_SPEEDTEST_10MS.csv' # PSS NID2=1
input_file = '/home/chatchamon/workarea/IQ_constellation/iq_python/input/IQDataFile_SPEEDTEST.csv'
# input_file = '/home/chatchamon/workarea/IQ_constellation/iq_python/input/IQDataFile_SPOTIFY.csv'  # PSS NID2=1
# input_file = '/home/chatchamon/workarea/IQ_constellation/iq_python/input/IQDataFile_SPOTIFY2.csv'

In [3]:
## read csv input file
with open(input_file, 'r') as f:
	reader = csv.reader(f)

	for _ in range(19):			# skip header
		next(reader)
	
	iq_val = []
	for row in reader:
		i_val = float(row[0])
		q_val = float(row[1])
		iq_val.append(complex(i_val, q_val))		# I+jQ
	
	waveform = np.array(iq_val[:230401])			# 10 ms
	# waveform = np.array(iq_val)			

print("len(waveform)", len(waveform))

len(waveform) 230401


In [4]:
delta_f = 0				# frequency offset
mu = 1					# for SCS
f = 1					# signal modulation

In [5]:
# scs = 30        # not sure
# syncNfft = 256 # minimum FFT size to cover SS burst
# syncSR = syncNfft * scs * 1e3     # move down, after get Nfft
# nrbSSB = 20
mu = 2

dec_factor = 8			# decimated factor, to downsample
waveform = scipy.signal.decimate(waveform, dec_factor, ftype='fir') 
waveform /= max(waveform.real.max(), waveform.imag.max()) # scale max amplitude to 1
fs = 23.04e6 // dec_factor		# sampling rate 23.04 MHz, to 2.88 MSPS

In [6]:
waveform =  waveform * np.exp(-1j*2*np.pi*delta_f/fs*(np.arange(len(waveform))))
np.random.seed(69) # to get reproducible noise
noise = ((np.random.rand(waveform.shape[0]) - 0.5) + 1j*(np.random.rand(waveform.shape[0]) - 0.5))*0.8
SNR = 10*np.log10((np.linalg.norm(waveform) / np.linalg.norm(noise)))
waveform += noise
print(f'SNR = {SNR} dB')
# print("waveform w/ noise:",waveform)
# print("len(waveform):", len(waveform))

SNR = -4.029200352636946 dB


In [7]:
# MATLAB: syncOfdmInfo = nrOFDMInfo(nrbSSB, scs, SampleRate = fs, Nfft = syncNfft)
carrier = nrCarrierConfig(NSizeGrid = 20, SubcarrierSpacing = 15 * 2**mu)
info = nrOFDMInfo(carrier)
Nfft = info['Nfft']     # Nfft = 512 for scs 30 kHz

In [8]:
pssIndices = np.arange((119-63), (119+64))      # same with nrPSSIndices()
 
## Frequency offset and PSS search
searchBW = 30*3       # don't know why
fshifts = np.linspace(-searchBW, searchBW, num=7) * 1e3 /2  # Half subcarrier step
print("fshifts", fshifts)

peak_value = np.zeros((fshifts.size, 3))
peak_index = np.zeros((fshifts.size, 3))
t = (np.arange(waveform.shape[0]) / fs).reshape(-1, 1)

# for fIdx in np.arange(len(fshifts)):

# test w/o loop
fIdx = 0
coarseFrequencyOffset = fshifts[fIdx]
rxWaveformFreqCorrected = waveform * np.exp(-1j*2*np.pi*coarseFrequencyOffset*t)

# waveform =  waveform * np.exp(-1j*2*np.pi*delta_f/fs*(np.arange(len(waveform))))

# in python file, below is in loop
# slotGrid = nrResourceGrid(carrier)
# slotGrid = slotGrid[:, 0]
# slotGrid[pssIndices] = nrPSS(current_NID2)
# [refWaveform, info] = nrOFDMModulate(carrier, slotGrid, SampleRate = fs)
# refWaveform = refWaveform[info['CyclicPrefixLengths'][0]:]; # remove CP

fshifts [-45000. -30000. -15000.      0.  15000.  30000.  45000.]


MemoryError: Unable to allocate 12.4 GiB for an array with shape (28801, 28801) and data type complex128